In [1]:
from Project.Database import Db
from Project._07OptimiseConsumption.emission_reduction import emission_reduction

In [2]:
NZERTF_optimisation, NZERTF_emission = emission_reduction(year=2)

In [3]:
NZERTF_optimisation

{'w.r':                Timestamp  Day  Hour  Load_LatentHeatWaterVolume  \
 0    2015-02-01 00:00:00    0     0                    0.027909   
 1    2015-02-01 01:00:00    0     1                    0.095475   
 2    2015-02-01 02:00:00    0     2                    0.163375   
 3    2015-02-01 03:00:00    0     3                    0.227850   
 4    2015-02-01 04:00:00    0     4                    0.291952   
 ...                  ...  ...   ...                         ...   
 8755 2016-01-31 19:00:00  364    19                    1.434078   
 8756 2016-01-31 20:00:00  364    20                    1.509165   
 8757 2016-01-31 21:00:00  364    21                    1.582327   
 8758 2016-01-31 22:00:00  364    22                    1.654222   
 8759 2016-01-31 23:00:00  364    23                    1.726894   
 
       Load_RefrigeratorTemp  Load_StatusBA1Lights  Load_StatusKitchenLightsA  \
 0                  4.957915                   0.0                   0.000000   
 1           

In [4]:
for key, value in NZERTF_optimisation.items():
    print(f'{key}: {value["Emission"]}')

w.r: 0       0.508536
1       0.727316
2       0.525879
3       0.508214
4       0.520379
          ...   
8755    1.642831
8756    1.360740
8757    1.407198
8758    0.939420
8759    0.602089
Name: Emission, Length: 8760, dtype: float64
w.o.r: 0       0.508536
1       0.727316
2       0.525879
3       0.508214
4       0.520379
          ...   
8755    1.462738
8756    1.278009
8757    1.407198
8758    0.926533
8759    0.600439
Name: Emission, Length: 8760, dtype: float64
m.a.o: 0       0.000000
1       0.000000
2       0.000000
3       0.000000
4       0.000000
          ...   
8755    0.000000
8756    0.000000
8757    0.033034
8758    0.117626
8759    0.117626
Name: Emission, Length: 8760, dtype: float64


In [5]:
NZERTF_emission

{'w.r': 8388.657846296948,
 'w.o.r': 8257.25142366896,
 'm.a.o': 377.724896268213,
 'm.a.u.o': 528.1785737581545}

year1= {'w.r': 6943587.30903113,
 'w.o.r': 6808693.4608922275,
 'm.a.u.o': 664541.8682200217,
 'm.a.o': 539569.892249094}

year2 = {'w.r': 8371902.086841607,
 'w.o.r': 8242621.19712289,
 'm.a.u.o': 511242.76383277134,
 'm.a.o': 603731.4005264738}

In [6]:
redundancy_reduction = NZERTF_emission['w.r'] - NZERTF_emission['w.o.r']
optimisation_reduction = NZERTF_emission['m.a.u.o'] - NZERTF_emission['m.a.o']
total_reduction = redundancy_reduction + optimisation_reduction

In [7]:
redundancy_reduction

131.40642262798792

In [8]:
optimisation_reduction

150.45367748994147

In [9]:
total_reduction

281.8601001179294

In [10]:
NZERTF_optimisation['m.a.o']

,Timestamp,Day,Hour,Load_StatusApplianceDishwasher,Load_StatusPlugLoadVacuum,Load_StatusClothesWasher,Load_StatusDryerPowerTotal,Load_StatusPlugLoadIron,Emission(g/Wh),Emission,OldLoad_StatusApplianceDishwasher,OldLoad_StatusPlugLoadVacuum,OldLoad_StatusClothesWasher,OldLoad_StatusDryerPowerTotal,OldLoad_StatusPlugLoadIron,OldEmission
0,2015-02-01 00:00:00,0,0,0.0000,0.0,0.0,0.0,0.0,0.734305,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000
1,2015-02-01 01:00:00,0,1,0.0000,0.0,0.0,0.0,0.0,0.752407,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000
2,2015-02-01 02:00:00,0,2,0.0000,0.0,0.0,0.0,0.0,0.737095,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000
3,2015-02-01 03:00:00,0,3,0.0000,0.0,0.0,0.0,0.0,0.733298,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000
4,2015-02-01 04:00:00,0,4,0.0000,0.0,0.0,0.0,0.0,0.731043,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,2016-01-31 19:00:00,364,19,0.0000,0.0,0.0,0.0,0.0,0.684510,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000
8756,2016-01-31 20:00:00,364,20,0.0000,0.0,0.0,0.0,0.0,0.697113,0.000000,0.550000,0.0,0.0,0.0,0.0,0.108122
8757,2016-01-31 21:00:00,364,21,0.2167,0.0,0.0,0.0,0.0,0.544998,0.033034,1.000000,0.0,0.0,0.0,0.0,0.153689
8758,2016-01-31 22:00:00,364,22,1.0000,0.0,0.0,0.0,0.0,0.420529,0.117626,0.666667,0.0,0.0,0.0,0.0,0.079059


In [11]:
NZERTF_optimisation['w.r']['Consumption'].sum()-NZERTF_optimisation['w.o.r']['Consumption'].sum()

182974.74609446712

In [12]:
NZERTF_optimisation['w.r']['Consumption'].groupby(NZERTF_optimisation['w.r']['Consumption'].index % 24).mean()

0      698.788280
1      709.881802
2      702.950804
3      725.324320
4      772.492298
5      739.047235
6     2977.813052
7     1597.746481
8     1640.184101
9     1367.349349
10    1109.436437
11     559.195413
12    1260.300588
13     710.009025
14    1027.502203
15    1109.782531
16     665.280911
17     748.913165
18    2941.576315
19    2525.628646
20    2473.697660
21    2272.299162
22    1995.166929
23    1131.257300
Name: Consumption, dtype: float64

## Hours and kWh reduced by redundancy removal

In [13]:
house_df, meta = Db.load_data(year=2, hourly=False, consumption=True, production=False, meta=True, with_redundancy=True)

In [14]:
status_constraint = meta.loc[(lambda self:
                           (self["Parameter"] == "Status_OnOff") &
                           (~self.index.str.contains("SensHeat")) &
                           (self["Subsystem"] == "Loads"))].index.to_list()

In [15]:
before_red_remov = house_df[status_constraint].sum()/60
before_red_remov.sum()

35973.56666666667

In [16]:
after_red_remov = NZERTF_optimisation['w.o.r'][status_constraint].sum()

In [17]:
before_red_remov.sum()-after_red_remov.sum()

972.3236864233331

In [18]:
(before_red_remov.sum()-after_red_remov.sum())/before_red_remov.sum()*100

2.7028837463711772